In [69]:
import math
import os
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import TensorDataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import json
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder

# Hyperparameters

In [70]:
# Transformer HyperHPeters
# Only 4 nucleotides possible (ACGT)
ntokens_HP = 4
# Number of attention heads
nhead_HP = 8
# Output size of the heads, which 
# learn an embedding.
head_embedsize_HP = 4
# For some reason PyTorch needs us to do this manually
d_model_HP = head_embedsize_HP * nhead_HP
# Output dimension of the feedforward network
# Set to 1 since this is a regression task
dim_feedforward_HP = 1
# No dropout for now
dropout_HP = 0
# Standard stuff
activation_HP = "relu"
layer_norm_eps_HP = 1e-5
batch_first_HP = True
norm_first_HP = False

# Trainig and validation Hyperparameters
datasetsize_HP = 100
split_HP = 0.2
batchsize_HP = 100
batchsize_HP = min(batchsize_HP, (int)(datasetsize_HP*0.2))
nepochs_HP = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_HP = nn.MSELoss()
learnrate_HP = 1e-4

In [71]:
# Create transformer model
class RegressionTranformer(nn.Module):
    def __init__(self):
        super(RegressionTranformer, self).__init__()
        self.encoder_transformer = nn.TransformerEncoderLayer(
            d_model = d_model_HP,
            nhead = nhead_HP,
            dim_feedforward = dim_feedforward_HP,
            dropout = dropout_HP,
            activation = activation_HP,
            layer_norm_eps = layer_norm_eps_HP,
            batch_first = batch_first_HP,
            norm_first = norm_first_HP
        )
        self.embedder = nn.Embedding(ntokens_HP, d_model_HP)
        self.

        # ADD FORWARD LAYER -> ENCODER OUTPUT -> 1 FOR REGRES

    def forward(self, x):
        ŷ = self.embedder(x)
        ŷ = self.encoder_transformer(ŷ)
        return ŷ
        

In [83]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(10, 512)
out = encoder_layer(src)
print(out.shape)

torch.Size([10, 512])


# Importation and preprocessing

In [72]:
features_df = pd.read_json('data\strandenergylist.json')
energy_df = features_df[1]
energy_df = energy_df.head(datasetsize_HP)
features_df = features_df.head(datasetsize_HP)
features_df = features_df[0].str.split('', expand=True)
features_df = features_df.drop([0, 31], axis=1)
labels_df = features_df[1]

### Transformation into pytorch tensors

In [73]:
# Transform string features into as DNA tensor
dna = features_df.to_numpy()
for i in range(dna.shape[0]):
    for j in range(dna.shape[1]):
        if (dna[i,j] == 'A'): dna[i,j] = 0
        if (dna[i,j] == 'C'): dna[i,j] = 1
        if (dna[i,j] == 'G'): dna[i,j] = 2
        if (dna[i,j] == 'T'): dna[i,j] = 3
dna = dna.astype(int)
dna = torch.from_numpy(dna)
dna = F.one_hot(dna.to(torch.int64), num_classes=4).to(device)
dna = dna.reshape(datasetsize_HP, 120)
# Get free energy as pytorch tensor
energy = torch.tensor(energy_df.values).to(device)

### Creation of test and validation iterators

In [74]:
dataset = TensorDataset(dna, energy)
ntrain = (int) (split_HP * datasetsize_HP)
ntest = datasetsize_HP - ntrain
train_set, vali_set = torch.utils.data.random_split(dataset, [ntrain, ntest])

train_dataloader = DataLoader(train_set, batch_size=batchsize_HP, shuffle=True)
vali_dataload = DataLoader(vali_set, batch_size=batchsize_HP, shuffle=True)

# Train the Transformer

In [75]:
transformer = RegressionTranformer()
transformer = transformer.to(device)

transformer_optimizer = torch.optim.Adam(transformer.parameters(), lr=learnrate_HP)



def trainroutine():
    for epoch in range(nepochs_HP):
        running_loss = 0.0
        for i, (x_batch, y_batch) in enumerate(train_dataloader):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Forward propagation
            ŷ_batch = transformer(x_batch)
            
            # Backpropagation
            batch_loss = loss_HP(ŷ_batch, y_batch).backward()
            batch_loss.backward()
            transformer_optimizer.step()
            transformer_optimizer.zero_grad()

            # Print statistics
            running_loss += batch_loss.item()
            if i % 1000 == 999:
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 1000))
                running_loss = 0.0
trainroutine()

c:\ProgramData\miniconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 120, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (32) must match the size of tensor b (20) at non-singleton dimension 2

# Notes

- We could add a positionnal encoding (with
PositionalEncoding) (see https://pytorch.org/tutorials/beginner/transformer_tutorial.html)

Usually, embedding is done in order to compress the representation of words (loss of information).
In our case, their are 4 possible tokens, which means that no compression is needed.